RFM模型介紹

在眾多 CRM 客戶資料分析中，RFM 模型是被最常被廣泛提到的，其中的三大要素為 R(Recency)、F(Frequency)、M(Monetary)：

-最近一次消費 (Recency)
-消費頻率 (Frequency)
-消費金額 (Monetary)

RFM 模型能將顧客資料分類後，可以針對不同類型顧客提供給予不同優惠，不同產品組合或是進行類似受眾廣告投遞；進而節省行銷預算上不必要的浪費。

需準備的會員資料:
-OrderId：訂單編號
-order_date：訂單日期
-TotalCharges：訂單花費

In [130]:
import numpy as np
import pandas as pd

In [152]:
rfmTable = pd.read_csv('C:/Users/USER/Desktop/Github/Python Project/RFM_table.csv')
rfmTable

#已先用R語言進行前處理
#時間範圍設定在1997/01/01~1998/01/01


,ID,MaxTransDate,Amount,Recency,Frequency,Monetary
0,1,1997-01-01,11.77,365,1,11.770000
1,2,1997-01-12,89.00,354,2,44.500000
2,3,1997-11-25,139.47,37,5,27.894000
3,4,1997-12-12,100.50,20,4,25.125000
4,5,1997-12-12,348.14,20,10,34.814000
...,...,...,...,...,...,...
23565,23566,1997-03-25,36.00,282,1,36.000000
23566,23567,1997-03-25,20.97,282,1,20.970000
23567,23568,1997-04-22,121.70,254,3,40.566667
23568,23569,1997-03-25,25.74,282,1,25.740000


-最近一次消費 (Recency):幾天前

-消費頻率 (Frequency):觀察期內有幾筆訂單(幾次消費)

-消費金額 (Monetary:觀察期內的訂單總額(金額)

設定RFM級距

In [153]:
quantile_r = rfmTable["Recency"].quantile(q=[0.25,0.5,0.75])

quantile_f = rfmTable["Frequency"].quantile(q=[0.25,0.8,0.95])

quantile_m = rfmTable["Monetary"].quantile(q=[0.25,0.5,0.85])

In [154]:
quantile_r

0.25    159.0
0.50    297.0
0.75    328.0
Name: Recency, dtype: float64

In [155]:
quantile_f

0.25    1.0
0.80    3.0
0.95    7.0
Name: Frequency, dtype: float64

In [156]:
quantile_m

0.25    14.96000
0.50    24.85125
0.85    51.73300
Name: Monetary, dtype: float64

In [157]:
quantiles = {"Recency" : quantile_r.to_dict() , 
             "Frequency" : quantile_f.to_dict(),
             "Monetary" : quantile_m.to_dict()}
quantiles

{'Recency': {0.25: 159.0, 0.5: 297.0, 0.75: 328.0},
 'Frequency': {0.25: 1.0, 0.8: 3.0, 0.95: 7.0},
 'Monetary': {0.25: 14.96, 0.5: 24.85125, 0.85: 51.73299999999996}}

In [158]:
rfmSegmentation = rfmTable

#最近一次消費:分為4個級距
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

    
#消費頻率:分為4個級距
def FClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.80]:
        return 2
    elif x <= d[p][0.95]: 
        return 3
    else:
        return 4
    
#消費總額:分為4個級距
def MClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.85]: 
        return 3
    else:
        return 4

In [159]:
rfmSegmentation

,ID,MaxTransDate,Amount,Recency,Frequency,Monetary
0,1,1997-01-01,11.77,365,1,11.770000
1,2,1997-01-12,89.00,354,2,44.500000
2,3,1997-11-25,139.47,37,5,27.894000
3,4,1997-12-12,100.50,20,4,25.125000
4,5,1997-12-12,348.14,20,10,34.814000
...,...,...,...,...,...,...
23565,23566,1997-03-25,36.00,282,1,36.000000
23566,23567,1997-03-25,20.97,282,1,20.970000
23567,23568,1997-04-22,121.70,254,3,40.566667
23568,23569,1997-03-25,25.74,282,1,25.740000


In [160]:
#計算分數:

rfmSegmentation['R_Quartile'] = rfmSegmentation['Recency'].apply(RClass, 
                                                                 args=('Recency',quantiles))

rfmSegmentation

,ID,MaxTransDate,Amount,Recency,Frequency,Monetary,R_Quartile
0,1,1997-01-01,11.77,365,1,11.770000,1
1,2,1997-01-12,89.00,354,2,44.500000,1
2,3,1997-11-25,139.47,37,5,27.894000,4
3,4,1997-12-12,100.50,20,4,25.125000,4
4,5,1997-12-12,348.14,20,10,34.814000,4
...,...,...,...,...,...,...,...
23565,23566,1997-03-25,36.00,282,1,36.000000,3
23566,23567,1997-03-25,20.97,282,1,20.970000,3
23567,23568,1997-04-22,121.70,254,3,40.566667,3
23568,23569,1997-03-25,25.74,282,1,25.740000,3


In [161]:
rfmSegmentation['F_Quartile'] = rfmSegmentation['Frequency'].apply(FClass, 
                                                                   args=('Frequency',quantiles))

rfmSegmentation

,ID,MaxTransDate,Amount,Recency,Frequency,Monetary,R_Quartile,F_Quartile
0,1,1997-01-01,11.77,365,1,11.770000,1,1
1,2,1997-01-12,89.00,354,2,44.500000,1,2
2,3,1997-11-25,139.47,37,5,27.894000,4,3
3,4,1997-12-12,100.50,20,4,25.125000,4,3
4,5,1997-12-12,348.14,20,10,34.814000,4,4
...,...,...,...,...,...,...,...,...
23565,23566,1997-03-25,36.00,282,1,36.000000,3,1
23566,23567,1997-03-25,20.97,282,1,20.970000,3,1
23567,23568,1997-04-22,121.70,254,3,40.566667,3,2
23568,23569,1997-03-25,25.74,282,1,25.740000,3,1


In [162]:
rfmSegmentation['M_Quartile'] = rfmSegmentation['Monetary'].apply(MClass,
                                                                  args=('Monetary',quantiles))

rfmSegmentation

,ID,MaxTransDate,Amount,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
0,1,1997-01-01,11.77,365,1,11.770000,1,1,1
1,2,1997-01-12,89.00,354,2,44.500000,1,2,3
2,3,1997-11-25,139.47,37,5,27.894000,4,3,3
3,4,1997-12-12,100.50,20,4,25.125000,4,3,3
4,5,1997-12-12,348.14,20,10,34.814000,4,4,3
...,...,...,...,...,...,...,...,...,...
23565,23566,1997-03-25,36.00,282,1,36.000000,3,1,3
23566,23567,1997-03-25,20.97,282,1,20.970000,3,1,2
23567,23568,1997-04-22,121.70,254,3,40.566667,3,2,3
23568,23569,1997-03-25,25.74,282,1,25.740000,3,1,3
